In [1]:
# Set the current working directory to the root directory
import os
import pickle as pkl
import random

from copy import deepcopy
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
os.chdir('../')

In [2]:
from classes.slicedkex import SlicedKex
from classes.smartkex import SmartKex
from classes.metakex import MetaKex
from classes.headerkex import HeaderKex
from classes.graphkex import GraphKex

from functions.utils import get_dataset_file_paths, generate_dataset
from functions.notebook_utils import merge_training_data_dicts

In [3]:
train_root = '../new'
test_root = '../validation/'
train_heap_paths, train_json_paths = get_dataset_file_paths(train_root)
test_heap_paths, test_json_paths = get_dataset_file_paths(test_root)

In [4]:
training_sizes = [500, 1000, 2000, 3000, 4000, 5000, 7500, 10000, 20000, 30000, 40000, 50000, 70000, 90000]

results = dict()
algorithms = ['slicedkex', 'smartkex', 'metakex', 'headerkex', 'graphkex']
classes = [SlicedKex, SmartKex, MetaKex, HeaderKex, GraphKex]
feature_vector_sizes = [128, 128, 176, 136, 9]

In [5]:
def generate_all_data(heap_paths, json_paths, algorithms, classes, feature_vector_sizes, train_subset=True, block_size=1000):
    test_dataset_dict = dict()
    for idx, algorithm in enumerate(algorithms):
        print(f'{datetime.now()}:Generating dataset for {algorithms[idx]}')
        test_dataset, test_labels = generate_dataset(heap_paths=heap_paths,
                                                     json_paths=json_paths,
                                                     obj_class=classes[idx], block_size=block_size,
                                                     train_subset=train_subset, feature_vector_size=feature_vector_sizes[idx])
        test_dataset_dict[algorithms[idx]] = [deepcopy(test_dataset), deepcopy(test_labels)]

    return test_dataset_dict


In [6]:
def train_all_algorithms(dataset_dict, seed, algorithms, classes, feature_vector_sizes, test_dataset_dict, results_dict):

    metrics = ['accuracy', 'precision', 'recall', 'f1']
    print(f'{datetime.now()}:Current Seed: {seed}')
    for idx, algorithm in enumerate(algorithms):

        clf = RandomForestClassifier(random_state=seed)
        dataset, labels = dataset_dict.get(algorithm)
        if len(labels) == 0:
            continue
        print(f'{datetime.now()}: Training {algorithms[idx]}')
        clf.fit(dataset, labels)

        test_dataset, test_labels = test_dataset_dict.get(algorithm)
        # For methods that use pointers, sometimes we would not sample heap files that have the relevant metadata.
        # This would cause the files to be 0, and this if condition addresses that scenario.
        if len(test_labels) == 0:
            algorithm_result = results_dict.get(algorithm, dict())

            accuracy_list = algorithm_result.get(metrics[0], [])
            accuracy_list.append(0.0)
            algorithm_result[metrics[0]] = accuracy_list

            precision_list = algorithm_result.get(metrics[1], [])
            precision_list.append(0.0)
            algorithm_result[metrics[1]] = precision_list

            recall_list = algorithm_result.get(metrics[2], [])
            recall_list.append(0.0)
            algorithm_result[metrics[2]] = recall_list

            f1_list = algorithm_result.get(metrics[3], [])
            f1_list.append(0.0)
            algorithm_result[metrics[3]] = f1_list
            continue
        print(f'{datetime.now()}: Predicting for {algorithms[idx]}')
        y_pred = clf.predict(test_dataset)
        precision = precision_score(y_true=test_labels, y_pred=y_pred)
        recall = recall_score(y_true=test_labels, y_pred=y_pred)
        f1_measure = f1_score(y_true=test_labels, y_pred=y_pred)
        accuracy = accuracy_score(y_true=test_labels, y_pred=y_pred)

        print(f'{datetime.now()}:F1-Measure for {algorithm} for dataset: {f1_measure}')

        algorithm_result = results_dict.get(algorithm, dict())

        accuracy_list = algorithm_result.get(metrics[0], [])
        accuracy_list.append(accuracy)
        algorithm_result[metrics[0]] = accuracy_list

        precision_list = algorithm_result.get(metrics[1], [])
        precision_list.append(precision)
        algorithm_result[metrics[1]] = precision_list

        recall_list = algorithm_result.get(metrics[2], [])
        recall_list.append(recall)
        algorithm_result[metrics[2]] = recall_list
        
        f1_list = algorithm_result.get(metrics[3], [])
        f1_list.append(0.0)
        algorithm_result[metrics[3]] = f1_list
        
        results_dict[algorithm] = algorithm_result

    return results_dict


In [7]:
# We set the block size at 16k because there are only 15000 files in the path for testing.
test_dataset_dict = generate_all_data(heap_paths=test_heap_paths, json_paths=test_json_paths, algorithms=algorithms,
                                      classes=classes, feature_vector_sizes=feature_vector_sizes, block_size=16000,
                                      train_subset=False)

2024-12-13 12:49:21.562925:Generating dataset for slicedkex


Data Files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15332/15332 [01:29<00:00, 170.88it/s]


Total files found: 15332
2024-12-13 13:08:34.800074:Generating dataset for smartkex


Data Files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15332/15332 [05:02<00:00, 50.66it/s]


Total files found: 15332
2024-12-13 13:21:49.654760:Generating dataset for metakex


...

Total files found: 15332
2024-12-13 13:37:21.077408:Generating dataset for headerkex


Data Files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15332/15332 [05:07<00:00, 49.88it/s]


Total files found: 15332
2024-12-13 13:50:36.940489:Generating dataset for graphkex


Data Files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15332/15332 [13:55<00:00, 18.35it/s]


Total files found: 12523


In [ ]:
prev_training_size = 0
results_dict = dict()
training_data_dict = dict()
# Set a constant seed for every random forest
for training_size in training_sizes:
    # print('Current training size:%d' % training_size)
    print('-----------------------------------------------------------------------------------')
    print(f'Current training size:{training_size}: Previous training size:{prev_training_size}')
    seed = random.randint(0, 65536)
    # Generate the training data only unique to this iteration
    curr_training_data_dict = generate_all_data(heap_paths=train_heap_paths[prev_training_size:training_size],
                                                json_paths=train_json_paths[prev_training_size:training_size],
                                                feature_vector_sizes=feature_vector_sizes, classes=classes,
                                                train_subset=False, block_size=training_size-prev_training_size,
                                                algorithms=algorithms)
    prev_training_size = training_size
    
    # Merge it to the previous dataset. This is for efficiency and not recomputing all the previous training data.
    training_data_dict = merge_training_data_dicts(training_data_dict, curr_training_data_dict)
    
    results_dict = train_all_algorithms(dataset_dict=training_data_dict, seed=seed, algorithms=algorithms,
                                        feature_vector_sizes=feature_vector_sizes, classes=classes, 
                                        test_dataset_dict=test_dataset_dict, results_dict=results_dict)
    with open('combined_results_short.pkl', 'wb') as fp:
        pkl.dump(results_dict, fp, protocol=pkl.HIGHEST_PROTOCOL)

-----------------------------------------------------------------------------------
Current training size:500: Previous training size:0
2024-12-13 14:06:53.585274:Generating dataset for slicedkex


Data Files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:02<00:00, 182.95it/s]


Total files found: 500
2024-12-13 14:07:18.666268:Generating dataset for smartkex


Data Files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:07<00:00, 68.10it/s]


Total files found: 500
2024-12-13 14:07:34.237842:Generating dataset for metakex


...

Total files found: 16760
2025-01-03 11:39:39.005119:Generating dataset for headerkex


Data Files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16760/16760 [07:03<00:00, 39.53it/s]


Total files found: 16760
2025-01-03 12:02:47.914728:Generating dataset for graphkex


Data Files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16760/16760 [22:19<00:00, 12.51it/s]


Total files found: 14629
2025-01-03 19:17:27.834392:Current Seed: 20436
2025-01-03 19:17:27.838206: Training slicedkex
